In [1]:
import http.server
import socketserver
import json
import time

In [2]:
current_time_millis = lambda: int(round(time.time() * 1000))

In [3]:
path_public_rsa = 'public_rsa'
path_get_file = 'get_file'
path_session_key = 'session_key'
path_login = 'login'
key_public_exponent = 'exponent'
key_modulus = 'modulus'
key_filename = 'file'
key_username = 'username'
key_password = 'password'
key_error = 'error'
key_content = 'content'
session_key_expiration_period = 1 * 60 * 1000 # 1 minute

class SessionKey:
    
    def __init__(self, key):
        self.key = key
        self.created = current_time_millis()
    
    def is_expired(self):
        return current_time_millis() - self.created > session_key_expiration_period
    
class DataHolder:
    
    def __init__(self):
        self.session_key = None
        self.public_exponent = None
        self.modulus = None
        
    def check_username_and_password(self, username, password):
        # todo add logic. possible answers: OK, WRONG_USERNAME, WRONG_PASSWORD, ERROR
        return 'OK'
    
    def file_exists(self, filename):
        # todo add logic
        return True
    
    def get_file_response(self, filename):
        response = {}
        if self.session_key is None or self.session_key.is_expired():
            response[key_error] = 'session_key_expired'
        elif not data_holder.file_exists(filename):
            response[key_error] = 'no_file'
        else:
            response[key_content] = 'file content' # todo replace with real encrypted content:
            #content in json format:
            # {
            #    "text": encrypted text,
            #    "iv": iv used for aes encryption
            #}
        return json.dumps(response)

In [4]:
data_holder = DataHolder()

class MyHTTPRequestHandler(http.server.BaseHTTPRequestHandler):

    def do_GET(self):
        split_by_question = self.path.split('?')
        path = split_by_question[0]
        argument_dict = {}
        if len(split_by_question) > 1:
            argument_pairs = split_by_question[1].split('&')
            for pair in argument_pairs:
                key_and_value = pair.split('=')
                argument_dict[key_and_value[0]] = key_and_value[1]
        if (path[0] == '/'):
            path = path[1:]
        if path[-1] == '/':
            path = path[:-1]
        if path == path_get_file:
            self.handle_get_file(argument_dict)
        elif path == path_session_key:
            self.handle_session_key()
        else:
            self.send_response(404)
            self.end_headers()            

    def do_POST(self):
        split_by_question = self.path.split('?')
        path = split_by_question[0]
        if (path[0] == '/'):
            path = path[1:]
        if path[-1] == '/':
            path = path[:-1]
        content_len = int(self.headers.get('Content-Length'))
        post_body = self.rfile.read(content_len)
        if path == path_login:
            self.handle_login(post_body)
        elif path == path_public_rsa:
            self.handle_public_rsa(post_body)
        else:
            self.send_response(404)
            self.end_headers()            

    def handle_login(self, post_body):
        print('received post:', post_body)
        try:
            post_json = json.loads(post_body)
            username = post_json[key_username]
            password = post_json[key_password]
            result = data_holder.check_username_and_password(username, password)
            print('username:', username, 'password:', password)
            self.send_response(200)
            self.end_headers()            
            self.wfile.write(bytearray(result, 'utf8'))
        except Exception as e:
            print(e)
            self.send_response(413)
            self.end_headers()                        

    def handle_public_rsa(self, post_body):
        try:
            post_json = json.loads(post_body)
            data_holder.public_exponent = post_json[key_public_exponent]
            data_holder.modulus = post_json[key_modulus]
            # todo save public key in case server is restarted
            print('handle_public_rsa: exp:', data_holder.public_exponent, ', modulus:', data_holder.modulus)
            self.send_response(200)
            self.end_headers()     
        except Exception as e:
            print(e)
            self.send_response(413)
            self.end_headers()     

    def handle_get_file(self, arguments):
        try:
            filename = arguments[key_filename]
            print('handle_get_file: ', filename)
            response = data_holder.get_file_response(filename)
            self.send_response(200)
            self.end_headers()    
            self.wfile.write(bytearray(response, 'utf8'))
        except Exception as e:
            print(e)
            self.send_response(413)
            self.end_headers()    

    def handle_session_key(self):
        print('handle_session_key')
        try:
            # todo send this key to client (encrypted with RSA)
            data_holder.session_key = SessionKey("todo: generate random key")
            self.send_response(200)
            self.end_headers()                    
        except Exception as e:
            print(e)
            self.send_response(413)
            self.end_headers()                    

In [ ]:
port = 8000
Handler = http.server.SimpleHTTPRequestHandler

httpd = http.server.HTTPServer(('localhost', port), MyHTTPRequestHandler)
print("serving at port", port)
httpd.serve_forever()

serving at port 8000
handle_get_file:  hhh


127.0.0.1 - - [22/Oct/2019 21:54:19] "GET /get_file?file=hhh HTTP/1.1" 200 -


handle_session_key


127.0.0.1 - - [22/Oct/2019 21:54:24] "GET /session_key HTTP/1.1" 200 -


handle_get_file:  uuu


127.0.0.1 - - [22/Oct/2019 21:54:28] "GET /get_file?file=uuu HTTP/1.1" 200 -
